In [0]:
/*
===============================================================================
Silver Layer DDL (Databricks / Unity Catalog / Delta)
===============================================================================
Purpose:
  - Create typed Silver tables (empty).
  - Create reject (_rej_*) tables for quarantined rows.

Environment:
  Catalog: datawarehouse
  Schema : silver

Notes:
  - NVARCHAR -> STRING, DATETIME2 -> TIMESTAMP, GETDATE() -> current_timestamp()
  - Tables use Delta (USING DELTA).
===============================================================================
*/
USE CATALOG datawarehouse;
USE SCHEMA silver;

/* ================== TYPED SILVER TABLES (EMPTY) ================== */

CREATE OR REPLACE TABLE crm_cust_info (
  cst_id             INT,
  cst_key            STRING,
  cst_firstname      STRING,
  cst_lastname       STRING,
  cst_marital_status STRING,
  cst_gndr           STRING,
  cst_create_date    DATE,
  dwh_create_date    TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

CREATE OR REPLACE TABLE crm_prd_info (
  prd_id          INT,
  cat_id          STRING,   -- from silver.erp_px_cat_g1v2.id
  prd_key         STRING,
  prd_nm          STRING,
  prd_cost        INT,
  prd_line        STRING,
  prd_start_dt    DATE,     -- tweaked: DATE (not TIMESTAMP)
  prd_end_dt      DATE,     -- tweaked: DATE (not TIMESTAMP)
  dwh_create_date TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

CREATE OR REPLACE TABLE crm_sales_details (
  sls_ord_num     STRING,
  sls_prd_key     STRING,
  sls_cust_id     INT,
  sls_order_dt    DATE,
  sls_ship_dt     DATE,
  sls_due_dt      DATE,
  sls_sales       INT,
  sls_quantity    INT,
  sls_price       INT,
  dwh_create_date TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

CREATE OR REPLACE TABLE erp_loc_a101 (
  cid             STRING,
  cntry           STRING,
  dwh_create_date TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

CREATE OR REPLACE TABLE erp_cust_az12 (
  cid             STRING,
  bdate           DATE,
  gen             STRING,
  dwh_create_date TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

CREATE OR REPLACE TABLE erp_px_cat_g1v2 (
  id              STRING,
  cat             STRING,
  subcat          STRING,
  maintenance     STRING,
  dwh_create_date TIMESTAMP DEFAULT current_timestamp()
) USING DELTA TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

/* ================== REJECT (QUARANTINE) TABLES (EMPTY) ================== */
/* Schema mirrors raw Bronze columns + audit fields */

CREATE OR REPLACE TABLE _rej_crm_cust_info (
  cst_id              STRING,
  cst_key             STRING,
  cst_firstname       STRING,
  cst_lastname        STRING,
  cst_marital_status  STRING,
  cst_gndr            STRING,
  cst_create_date     STRING,
  error_reason        STRING,
  captured_ts         TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE _rej_crm_prd_info (
  prd_id       STRING,
  prd_key      STRING,
  prd_nm       STRING,
  prd_cost     STRING,
  prd_line     STRING,
  prd_start_dt STRING,
  prd_end_dt   STRING,
  error_reason STRING,
  captured_ts  TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE _rej_crm_sales_details (
  sls_ord_num  STRING,
  sls_prd_key  STRING,
  sls_cust_id  STRING,
  sls_order_dt STRING,
  sls_ship_dt  STRING,
  sls_due_dt   STRING,
  sls_sales    STRING,
  sls_quantity STRING,
  sls_price    STRING,
  error_reason STRING,
  captured_ts  TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE _rej_erp_loc_a101 (
  cid          STRING,
  cntry        STRING,
  error_reason STRING,
  captured_ts  TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE _rej_erp_cust_az12 (
  cid          STRING,
  bdate        STRING,
  gen          STRING,
  error_reason STRING,
  captured_ts  TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE _rej_erp_px_cat_g1v2 (
  id           STRING,
  cat          STRING,
  subcat       STRING,
  maintenance  STRING,
  error_reason STRING,
  captured_ts  TIMESTAMP
) USING DELTA;
